# 2016-01-15の着順と特徴量の相関

対象ファイル:
- data/previews/2016/01/15.csv
- data/results/2016/01/15.csv

目的: プレビュー情報(艇番ごとの情報 + 気象など)と着順の相関を確認する。


注意:
- 着順は小さいほど良いので、相関が負なら着順が良くなる方向。
- 結果データに欠損がある行は除外する。


In [ ]:
from pathlib import Path
import pandas as pd

cwd = Path.cwd()
repo_root = cwd if (cwd / 'data').exists() else cwd.parent.parent
preview_path = repo_root / 'data' / 'previews' / '2016' / '01' / '15.csv'
result_path = repo_root / 'data' / 'results' / '2016' / '01' / '15.csv'

preview_path.exists(), result_path.exists()


In [ ]:
preview = pd.read_csv(preview_path)
results = pd.read_csv(result_path)

preview.shape, results.shape


In [ ]:
race_cols = [
    'レースコード', 'タイトル', 'レース日', 'レース場', 'レース回',
    '風速(m)', '風向', '波の高さ(cm)', '天候', '気温(℃)', '水温(℃)'
]
boat_features = [
    '艇番', 'コース', '体重(kg)', '体重調整(kg)', '展示タイム', 'チルト調整', 'スタート展示'
]

boat_frames = []
for b in range(1, 7):
    rename = {f'艇{b}_{col}': col for col in boat_features}
    cols = race_cols + list(rename.keys())
    tmp = preview[cols].rename(columns=rename)
    tmp['艇'] = b
    boat_frames.append(tmp)

preview_long = pd.concat(boat_frames, ignore_index=True)
preview_long.head()


In [ ]:
result_frames = []
for order in range(1, 7):
    col = f'{order}着_艇番'
    if col in results.columns:
        tmp = results[['レースコード', col]].copy()
        tmp = tmp.rename(columns={col: '艇番'})
        tmp['着順'] = order
        result_frames.append(tmp)

result_long = pd.concat(result_frames, ignore_index=True)
result_long = result_long.dropna(subset=['艇番'])
result_long.head()


In [ ]:
merged = preview_long.merge(result_long, on=['レースコード', '艇番'], how='left')
merged['レース回_num'] = (
    merged['レース回']
    .astype(str)
    .str.extract(r'(\d+)')
    .astype(float)
)

merged[['レースコード', '艇番', '着順']].head()


In [ ]:
merged['着順'].isna().mean()


In [ ]:
feature_cols = [
    'レース場', 'レース回_num', '風速(m)', '風向', '波の高さ(cm)', '天候', '気温(℃)', '水温(℃)',
    '艇', '艇番', 'コース', '体重(kg)', '体重調整(kg)', '展示タイム', 'チルト調整', 'スタート展示'
]
feature_cols = [c for c in feature_cols if c in merged.columns]

numeric = merged[feature_cols + ['着順']].apply(pd.to_numeric, errors='coerce')
numeric = numeric.dropna(subset=['着順'])

pearson = numeric[feature_cols].corrwith(numeric['着順'])
spearman = numeric[feature_cols + ['着順']].corr(method='spearman')['着順'].drop('着順')

corr = (
    pd.DataFrame({
        'pearson': pearson,
        'spearman': spearman,
    })
    .assign(
        abs_pearson=lambda df: df['pearson'].abs(),
        abs_spearman=lambda df: df['spearman'].abs(),
    )
    .sort_values('abs_spearman', ascending=False)
)

corr


In [ ]:
import matplotlib.pyplot as plt

top = corr.dropna().head(10).sort_values('spearman')
ax = top['spearman'].plot(kind='barh', figsize=(7, 4), title='着順とのSpearman相関 (Top10)')
ax.axvline(0, color='black', linewidth=1)
plt.tight_layout()


次の一歩(特徴量エンジニアリングの例):
- レース内での相対値(展示タイムの平均との差や順位)を作る。
- 風向×コース、艇番×コースなどの交互作用を試す。
- コース別のスタート展示平均との差を使う。
